# Using the US EPA AQS API

This example code, uses the [US EPA Air Quality System API](https://aqs.epa.gov/aqsweb/documents/data_api.html) to retreive and save historical Air Quality Data. 

Please refer to the [lecture](https://isat-drg.github.io/ISAT_420/EnvironmentalData/3_APIs/workingWithAPIs.html) for further detail. 

In [1]:
import requests

In [78]:
import yaml
with open('C:/Users/tobia/.epaapi.yml', 'r') as file:
    api_info = yaml.load(file, Loader=yaml.FullLoader)


In [83]:
#https://aqs.epa.gov/data/api/list/parametersByClass?email=test@aqs.api&key=test&pc=CRITERIA
parameters  = parameters = {'email': 'gerkentx@jmu.edu',
            'key':  api_info['key'],
            'pc':"CRITERIA"}
r=requests.get("https://aqs.epa.gov/data/api/list/parametersByClass", params = parameters)
r.json()

{'Header': [{'status': 'Success',
   'request_time': '2024-02-20T12:48:17-05:00',
   'url': 'https://aqs.epa.gov/data/api/list/parametersByClass?email=gerkentx%40jmu.edu&key=ochreswift41&pc=CRITERIA',
   'rows': 8}],
 'Data': [{'code': '14129', 'value_represented': 'Lead (TSP) LC'},
  {'code': '42101', 'value_represented': 'Carbon monoxide'},
  {'code': '42401', 'value_represented': 'Sulfur dioxide'},
  {'code': '42602', 'value_represented': 'Nitrogen dioxide (NO2)'},
  {'code': '44201', 'value_represented': 'Ozone'},
  {'code': '81102', 'value_represented': 'PM10 Total 0-10um STP'},
  {'code': '85129', 'value_represented': 'Lead PM10 LC FRM/FEM'},
  {'code': '88101', 'value_represented': 'PM2.5 - Local Conditions'}]}

In [85]:
parameters  = parameters = {'email': 'gerkentx@jmu.edu',
            'key':  api_info['key'],
            'bdate': 20200101,
            'edate': 20200101,
            'state':"06",
             "county":"037",               
            'param':"44201"            }
            
r=requests.get("https://aqs.epa.gov/data/api/monitors/byCounty", params = parameters)

data = r.json()
df = pd.json_normalize(data['Data'])
df.head()

,state_code,county_code,site_number,parameter_code,poc,parameter_name,open_date,close_date,concurred_exclusions,dominant_source,...,address,state_name,county_name,city_name,cbsa_code,cbsa_name,csa_code,csa_name,tribal_code,tribe_name
0,06,037,0002,44201,1,Ozone,1978-06-01,2022-09-29,None,AREA,...,"803 N. LOREN AVE., AZUSA",California,Los Angeles,Azusa,31080,"Los Angeles-Long Beach-Anaheim, CA",348,"Los Angeles-Long Beach, CA",None,None
1,06,037,0016,44201,1,Ozone,1980-01-01,None,None,None,...,"840 LAUREL, GLENDORA",California,Los Angeles,Glendora,31080,"Los Angeles-Long Beach-Anaheim, CA",348,"Los Angeles-Long Beach, CA",None,None
2,06,037,0113,44201,1,Ozone,1985-01-01,None,None,None,...,"VA HOSPITAL, WEST LOS ANGELES",California,Los Angeles,West Los Angeles,31080,"Los Angeles-Long Beach-Anaheim, CA",348,"Los Angeles-Long Beach, CA",None,None
3,06,037,1201,44201,1,Ozone,1978-01-01,None,None,None,...,"18330 GAULT ST., RESEDA",California,Los Angeles,Reseda,31080,"Los Angeles-Long Beach-Anaheim, CA",348,"Los Angeles-Long Beach, CA",None,None
4,06,037,1701,44201,1,Ozone,1978-01-01,None,None,None,...,"924 N. GAREY AVE., POMONA",California,Los Angeles,Pomona,31080,"Los Angeles-Long Beach-Anaheim, CA",348,"Los Angeles-Long Beach, CA",None,None
5,06,037,6012,44201,1,Ozone,2001-02-28,None,None,None,...,"22224 PLACERITA CANYON RD, SANTA CLARITA",California,Los Angeles,Santa Clarita,31080,"Los Angeles-Long Beach-Anaheim, CA",348,"Los Angeles-Long Beach, CA",None,None
6,06,037,9033,44201,1,Ozone,2001-11-01,None,None,None,...,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",348,"Los Angeles-Long Beach, CA",None,None
7,06,037,2005,44201,1,Ozone,1982-03-01,None,None,None,...,"752 S. WILSON AVE., PASADENA",California,Los Angeles,Pasadena,31080,"Los Angeles-Long Beach-Anaheim, CA",348,"Los Angeles-Long Beach, CA",None,None
8,06,037,5005,44201,1,Ozone,2004-04-12,2021-09-16,None,MOBILE,...,7201 W. WESTCHESTER PARKWAY,California,Los Angeles,Los Angeles,31080,"Los Angeles-Long Beach-Anaheim, CA",348,"Los Angeles-Long Beach, CA",None,None
9,06,037,1602,44201,1,Ozone,2005-09-12,None,None,None,...,"4144 SAN GABRIEL RIVER PKWY, PICO RIVERA",California,Los Angeles,Pico Rivera,31080,"Los Angeles-Long Beach-Anaheim, CA",348,"Los Angeles-Long Beach, CA",None,None


In [86]:
m = folium.Map(location=[34.052235, -118.2436831])
for index, row in df.iterrows():
    folium.Marker(location=[row.latitude,row.longitude], popup =  row.local_site_name + ' ' + row.site_number).add_to(m)
m

In [88]:
sites  = {'Compton': 1302, 'Pomona': 1701, 'Pasadena':2005}
import time
for key, value in sites.items():
        
        parameters  = {'email': 'gerkentx@jmu.edu',
            'key':  api_info['key'],
            'bdate': 20200101,
            'edate': 20201231,
            'state':"06",
            'county':'037', 
            'site': str(value),                        
            'param': "42602,44201,81102,88101" }
        
        r=requests.get("https://aqs.epa.gov/data/api/sampleData/bySite", params = parameters)

        data = r.json()
        dfSample = json_normalize(data['Data'])
        dfSample['DateTime']=pd.to_datetime(dfSample.date_local.values + ' ' + dfSample.time_local.values)
        dfSample = dfSample.set_index('DateTime')
        fname = f'../data/EPA_AQI_{key}_{value}_{parameters["bdate"]}_{parameters["edate"]}.csv'
        print(fname)
        time.sleep(5)
        dfSample.to_csv(fname)
        
dfSample

C:\Users\tobia\AppData\Local\Temp\ipykernel_17344\2517149916.py:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  dfSample = json_normalize(data['Data'])


../data/EPA_AQI_Compton_1302_20200101_20201231.csv


C:\Users\tobia\AppData\Local\Temp\ipykernel_17344\2517149916.py:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  dfSample = json_normalize(data['Data'])


../data/EPA_AQI_Pomona_1701_20200101_20201231.csv


C:\Users\tobia\AppData\Local\Temp\ipykernel_17344\2517149916.py:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  dfSample = json_normalize(data['Data'])


../data/EPA_AQI_Pasedenia_2005_20200101_20201231.csv


,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,date_local,...,detection_limit,uncertainty,qualifier,method_type,method,method_code,state,county,date_of_last_change,cbsa_code
DateTime,,,,,,,,,,,,,,,,,,,,,
2020-01-14 00:00:00,06,037,2005,42602,1,34.1326,-118.1272,WGS84,Nitrogen dioxide (NO2),2020-01-14,...,2.7,None,None,FRM,INSTRUMENTAL - GAS PHASE CHEMILUMINESCENCE,099,California,Los Angeles,2020-06-19,31080
2020-01-14 01:00:00,06,037,2005,42602,1,34.1326,-118.1272,WGS84,Nitrogen dioxide (NO2),2020-01-14,...,2.7,None,None,FRM,INSTRUMENTAL - GAS PHASE CHEMILUMINESCENCE,099,California,Los Angeles,2020-06-19,31080
2020-01-14 02:00:00,06,037,2005,42602,1,34.1326,-118.1272,WGS84,Nitrogen dioxide (NO2),2020-01-14,...,2.7,None,None,FRM,INSTRUMENTAL - GAS PHASE CHEMILUMINESCENCE,099,California,Los Angeles,2020-06-19,31080
2020-01-14 03:00:00,06,037,2005,42602,1,34.1326,-118.1272,WGS84,Nitrogen dioxide (NO2),2020-01-14,...,2.7,None,None,FRM,INSTRUMENTAL - GAS PHASE CHEMILUMINESCENCE,099,California,Los Angeles,2020-06-19,31080
2020-01-14 04:00:00,06,037,2005,42602,1,34.1326,-118.1272,WGS84,Nitrogen dioxide (NO2),2020-01-14,...,2.7,None,None,FRM,INSTRUMENTAL - GAS PHASE CHEMILUMINESCENCE,099,California,Los Angeles,2020-06-19,31080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-10 00:00:00,06,037,2005,88101,2,34.1326,-118.1272,WGS84,PM2.5 - Local Conditions,2020-01-10,...,2.0,None,None,FRM,Thermo Electron Model RAAS2.5-300 Sequential w...,155,California,Los Angeles,2020-05-28,31080
2020-01-16 00:00:00,06,037,2005,88101,2,34.1326,-118.1272,WGS84,PM2.5 - Local Conditions,2020-01-16,...,2.0,None,None,FRM,Thermo Electron Model RAAS2.5-300 Sequential w...,155,California,Los Angeles,2020-05-28,31080
2020-01-22 00:00:00,06,037,2005,88101,2,34.1326,-118.1272,WGS84,PM2.5 - Local Conditions,2020-01-22,...,2.0,None,None,FRM,Thermo Electron Model RAAS2.5-300 Sequential w...,155,California,Los Angeles,2020-05-28,31080


In [59]:
dfSample['parameter'].unique()

array(['Nitrogen dioxide (NO2)', 'Ozone', 'PM2.5 - Local Conditions'],
      dtype=object)

In [30]:
dfSample.columns

Index(['state_code', 'county_code', 'site_number', 'parameter_code', 'poc',
       'latitude', 'longitude', 'datum', 'parameter', 'date_local',
       'time_local', 'date_gmt', 'time_gmt', 'sample_measurement',
       'units_of_measure', 'units_of_measure_code', 'sample_duration',
       'sample_duration_code', 'sample_frequency', 'detection_limit',
       'uncertainty', 'qualifier', 'method_type', 'method', 'method_code',
       'state', 'county', 'date_of_last_change', 'cbsa_code'],
      dtype='object')